In [19]:
import os
import sys
import cv2
import tensorflow as tf
import numpy as np
from tensorflow import keras
from matplotlib import pyplot
from skimage import transform

In [20]:
#return labels and ids of images
directory = 'D:/Plant_Phenotyping_Datasets/Plant/Ara2013-RPi'
def labels_and_ids():
    
    label_col = 1
    id_col = 0
    file = open('D:\Plant_Phenotyping_Datasets\Plant\Ara2013-RPi\Metadata.csv', 'r' , encoding='utf-8-sig')
    labels = []
    ids = []

    for line in file:
        line = line.rstrip()
        temp = line.split(',')
        labels.append(temp[label_col])
        ids.append(temp[id_col])
    return labels,ids

In [21]:
#Data augmentation
def augment(image):
    image = segment(image)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image , [299,299])
    image = (image/255.0)
    image = tf.image.random_brightness(image, max_delta=0.5)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.5 )
    image = tf.image.flip_left_right(image)
    #image = tf.image.central_crop(image,central_fraction=0.5)
    return image
    

In [22]:
def display(a, b, title1 = "Original", title2 = "Edited"):
    pyplot.subplot(121), pyplot.imshow(a), pyplot.title(title1)
    pyplot.xticks([]), pyplot.yticks([])
    pyplot.subplot(122), pyplot.imshow(b), pyplot.title(title2)
    pyplot.xticks([]), pyplot.yticks([])
    pyplot.show()
#data segmentation
def segment(image):
    gray  = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    ret , thresh  = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    # Further noise removal
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

# sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    ret, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    display(image,thresh,'Original','Segmented')
    return thresh

In [ ]:
#loading dataset

X= []
Y = []
labels,ids = labels_and_ids()
for id in ids:
    failed = 0
    image_path = os.path.join(directory , id + '_rgb.png')
    #image_path = 'D:/Plant_Phenotyping_Datasets/Plant/Ara2013-RPi'+'/'+id+'.png'
    #print(image_path)
    print(image_path)
    
    img_arry = cv2.imread(image_path)
    image = augment(img_arry)
    #print(image)
    X.append(image)
    
for label in labels:
    Y.append(label)

In [23]:
X= np.asarray(X)
Y= np.asarray(Y).astype('float32')

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train , y_test = train_test_split(X,Y,test_size = 0.20)

#x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.20)

In [25]:
x_train = x_train.astype('float32')
#x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

In [26]:
x_train.shape
x_test.shape

(33, 299, 299, 3)

In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [28]:
from keras.applications.xception import Xception
from keras.models import Model

model = Xception(weights=None, include_top=True, input_shape=(299,299, 3))

x = model.get_layer(index=len(model.layers)-2).output
print("////////////////////X")
print(x)
x = Dense(1)(x)

model = Model(inputs=model.input, outputs=x)
model.summary()


hello
(3, 3, 3, 32)
hello
-0.13801311186847084
hello
0.13801311186847084
hello
[32]
hello
1
hello
[32]
hello
0
hello
[32]
hello
0
hello
[32]
hello
1
hello
(3, 3, 32, 64)
hello
-0.08333333333333333
hello
0.08333333333333333
hello
[64]
hello
1
hello
[64]
hello
0
hello
[64]
hello
0
hello
[64]
hello
1
hello
(1, 1, 64, 128)
hello
-0.1767766952966369
hello
0.1767766952966369
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(3, 3, 64, 1)
hello
-0.10127393670836667
hello
0.10127393670836667
hello
(1, 1, 64, 128)
hello
-0.1767766952966369
hello
0.1767766952966369
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(3, 3, 128, 1)
hello
-0.07188851546895894
hello
0.07188851546895894
hello
(1, 1, 128, 128)
hello
-0.15309310892394862
hello
0.15309310892394862
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(1, 1, 128, 256)
hello
-0.125
hello
0.125
hello
[256]
hello
1
hello
[256]
hello
0
hell

hello
[1000]
hello
0
hello
0
hello
0
hello
0
hello
0
hello
0
hello
0
////////////////////X
Tensor("avg_pool/Mean_1:0", shape=(None, 2048), dtype=float32)
hello
(2048, 1)
hello
-0.05411337807489408
hello
0.05411337807489408
hello
[1]
hello
0
hello
0
hello
0
hello
0
hello
0
hello
0
hello
0
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]            

Total params: 20,863,529
Trainable params: 20,809,001
Non-trainable params: 54,528
__________________________________________________________________________________________________


In [30]:
from keras.optimizers import RMSprop, SGD, Adam
opt = RMSprop(lr=0.0001)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
network_history = model.fit(x_train, y_train, batch_size=19, epochs=20, verbose=1)

hello
[]
hello
0
hello
[]
hello
0
hello
1
hello
[[[[-0.24406281 -0.1175824  -0.11518931]
   [-0.24354121 -0.1169956  -0.11460251]
   [-0.24038349 -0.11344315 -0.11105006]
   ...
   [-0.24103548 -0.11982384 -0.11907832]
   [-0.24182491 -0.12061327 -0.12026248]
   [-0.24195531 -0.12074366 -0.12045807]]

  [[-0.24390775 -0.11742733 -0.11503424]
   [-0.24339813 -0.11685252 -0.11445943]
   [-0.24031302 -0.11337269 -0.11097959]
   ...
   [-0.24070843 -0.11944109 -0.11869066]
   [-0.24151237 -0.12025957 -0.11987482]
   [-0.24164517 -0.12039477 -0.12007041]]

  [[-0.24239936 -0.11591895 -0.11352585]
   [-0.24200642 -0.11546081 -0.11306771]
   [-0.23962756 -0.11268724 -0.11029414]
   ...
   [-0.23752707 -0.11571805 -0.11491971]
   [-0.23847227 -0.11681904 -0.11610386]
   [-0.2386284  -0.1170009  -0.11629945]]

  ...

  [[-0.22867237 -0.10429946 -0.09874511]
   [-0.22871357 -0.10434067 -0.09878632]
   [-0.22896308 -0.10459017 -0.09903581]
   ...
   [-0.23213118 -0.11706056 -0.1249266 ]
   [-0.24

hello
1
hello
-1
hello
-1
hello
0
Epoch 1/20
hello
[]
hello
0
hello
0.0
hello
0.0001
hello
0.0
hello
0.9
hello
[3, 3, 3, 32]
hello
0
hello
[32]
hello
0
hello
[32]
hello
0
hello
[3, 3, 32, 64]
hello
0
hello
[64]
hello
0
hello
[64]
hello
0
hello
[3, 3, 64, 1]
hello
0
hello
[1, 1, 64, 128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
0
hello
[3, 3, 128, 1]
hello
0
hello
[1, 1, 128, 128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
0
hello
[1, 1, 64, 128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
0
hello
[3, 3, 128, 1]
hello
0
hello
[1, 1, 128, 256]
hello
0
hello
[256]
hello
0
hello
[256]
hello
0
hello
[3, 3, 256, 1]
hello
0
hello
[1, 1, 256, 256]
hello
0
hello
[256]
hello
0
hello
[256]
hello
0
hello
[1, 1, 128, 256]
hello
0
hello
[256]
hello
0
hello
[256]
hello
0
hello
[3, 3, 256, 1]
hello
0
hello
[1, 1, 256, 728]
hello
0
hello
[728]
hello
0
hello
[728]
hello
0
hello
[3, 3, 728, 1]
hello
0
hello
[1, 1, 728, 728]
hello
0
hello
[728]
hello
0
hello
[728]
hello
0
hello
[1, 1, 256, 728]


In [31]:
model.save('C:/Users/Mohsin/JupyterNotebook/SegmentedXceptionmodel.h5')

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
from tensorflow import keras
model  = keras.models.load_model('C:/Users/Mohsin/JupyterNotebook/Xceptionmodel.h5')
predictTrain = model.predict(x_train, verbose=1)
predictTrain = predictTrain.reshape(predictTrain.shape[0])
#predictTrain = predictTrain.astype('int32')
print(x_train.shape)
print(predictTrain.shape)
print(y_train.shape)
print("predict train")
print(np.round(predictTrain))
print("y train")
print(y_train)

hello
(3, 3, 3, 32)
hello
-0.13801311186847084
hello
0.13801311186847084
hello
[32]
hello
1
hello
[32]
hello
0
hello
[32]
hello
0
hello
[32]
hello
1
hello
(3, 3, 32, 64)
hello
-0.08333333333333333
hello
0.08333333333333333
hello
[64]
hello
1
hello
[64]
hello
0
hello
[64]
hello
0
hello
[64]
hello
1
hello
(3, 3, 64, 1)
hello
-0.10127393670836667
hello
0.10127393670836667
hello
(1, 1, 64, 128)
hello
-0.1767766952966369
hello
0.1767766952966369
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(3, 3, 128, 1)
hello
-0.07188851546895894
hello
0.07188851546895894
hello
(1, 1, 128, 128)
hello
-0.15309310892394862
hello
0.15309310892394862
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(1, 1, 64, 128)
hello
-0.1767766952966369
hello
0.1767766952966369
hello
[128]
hello
1
hello
[128]
hello
0
hello
[128]
hello
0
hello
[128]
hello
1
hello
(3, 3, 128, 1)
hello
-0.07188851546895894
hello
0.07188851546895894
hello
(1, 1, 128, 

hello
[[[[ 0.04306636 -0.09620272 -0.09121867 -0.00832027 -0.12947157
     0.10396687 -0.07005405  0.03781589  0.11293913 -0.05473
     0.13301569  0.11129199  0.10917905 -0.11516687 -0.05451032
    -0.05383585 -0.10581073 -0.0958116   0.09753619 -0.02288917
     0.08522421  0.11684275 -0.11939859  0.00711981  0.11084717
    -0.02102357  0.12757911 -0.07990081 -0.13746242  0.06754284
     0.06062248 -0.03971395]
   [-0.01561754 -0.06005869  0.11777735  0.08785315 -0.11050078
    -0.08356118  0.13914196  0.13366814 -0.01065602 -0.01134279
    -0.0246631  -0.02359222 -0.05820918 -0.0908189   0.08307043
    -0.0788793   0.12572773 -0.03935344 -0.09993806  0.04840017
     0.06021707  0.08206104 -0.07271085  0.04437618 -0.03800214
     0.09070674 -0.10367166 -0.01662275 -0.08130057 -0.04247459
     0.07906327  0.12847215]
   [ 0.03099379  0.02465784  0.11662494 -0.03606888 -0.12186651
     0.03744863 -0.05441876 -0.10793534 -0.10834184 -0.03017148
    -0.13062826  0.04399751 -0.01959156  0.

[0.000887   0.00120182 0.0008922  0.0008549  0.00115093 0.00203039
 0.00116442 0.00083768 0.00117311 0.00158765 0.00076104 0.00276177
 0.00098877 0.00082016 0.00075438 0.00099139 0.00093799 0.00068484
 0.00195433 0.0009993  0.00105074 0.00114085 0.00085727 0.00123413
 0.00078336 0.00104014 0.00219242 0.00102792 0.00101339 0.00138246
 0.00082419 0.00101966 0.0019642  0.00073702 0.00088762 0.00097435
 0.00218466 0.00154723 0.00234089 0.00087491 0.0013037  0.00194497
 0.00231877 0.00092766 0.00102326 0.00168964 0.0008925  0.00095025
 0.00094505 0.00105193 0.00078241 0.00104348 0.0010777  0.00094768
 0.00133526 0.00124793 0.00144972 0.00271649 0.00097091 0.00180006
 0.00103287 0.00174077 0.00091851 0.00224753 0.00104867 0.00090088
 0.00145384 0.00082946 0.00102723 0.00087606 0.00083171 0.00092976
 0.00082245 0.00121056 0.00172172 0.00167933 0.00088762 0.00087795
 0.00076674 0.00286339 0.00086007 0.00084953 0.00295248 0.00102769
 0.00110788 0.0012175  0.00116517 0.00084896 0.00097607 0.0014

[0.00327764 0.00580148 0.00218649 0.00357179 0.00259316 0.00727453
 0.00232557 0.00490648 0.01030251 0.0034946  0.0019925  0.00334711
 0.0028028  0.0028836  0.00247964 0.00346759 0.00314828 0.00243914
 0.00372606 0.00553841 0.0049687  0.00541461 0.00362007 0.00478247
 0.00302119 0.00233571 0.00290769 0.00359442 0.00507798 0.00420602
 0.00392182 0.00248626 0.00304922 0.00270486 0.00290331 0.00271187
 0.00434005 0.00265904 0.00316947 0.00209761 0.0040049  0.00556868
 0.00297392 0.00331643 0.00208937 0.00349019 0.00214629 0.00331969
 0.00256439 0.00247601 0.00314687 0.00294397 0.00346378 0.00250915
 0.00271967 0.00549639 0.00373714 0.00365249 0.00351273 0.00442861
 0.00243155 0.00376909 0.00314749 0.00636531 0.00258043 0.0022238
 0.0062386  0.0039502  0.00399899 0.00527356 0.00473076 0.00658246
 0.00277162 0.00325897 0.00293922 0.00308572 0.00236161 0.00288298
 0.00409952 0.00789897 0.00726421 0.003816   0.00326733 0.00311658
 0.00484148 0.00270726 0.00279206 0.00836167 0.00441951 0.00538

hello
[0.00076879 0.00139848 0.00065709 0.00101983 0.00073383 0.00084064
 0.00079836 0.00106617 0.0006678  0.00100655 0.00079257 0.00100016
 0.00072918 0.00064481 0.00078982 0.00079462 0.00080639 0.0006869
 0.00097237 0.00141388 0.000709   0.00075575 0.00068917 0.00064681
 0.00070915 0.00077483 0.00067163 0.00078148 0.0008967  0.00063282
 0.00080993 0.00083694 0.000803   0.00081429 0.00083628 0.00083661
 0.0007382  0.00098974 0.00071351 0.00075415 0.00068246 0.00081518
 0.0009446  0.00111852 0.00099455 0.00079432 0.00106607 0.00089591
 0.0009384  0.00089248 0.00095392 0.00069294 0.00105229 0.0008598
 0.0008076  0.00072043 0.00102062 0.00072845 0.00070706 0.00095351
 0.0010472  0.00080761 0.00115372 0.00069869 0.00075113 0.00094094
 0.00087965 0.00074221 0.000762   0.00089062 0.000887   0.00060709
 0.0006922  0.00082141 0.00068742 0.0007309  0.00097774 0.00077436
 0.00077523 0.00064292 0.00070164 0.00071427 0.00067609 0.00066043
 0.00089686 0.00078489 0.00082062 0.0006777  0.00101146 0.

...

[6.10838597e-08 6.96584834e-08 9.35727869e-08 5.29027311e-08
 3.95601170e-08 3.48606193e-08 3.21442513e-07 1.36325852e-07
 1.61794716e-07 1.00897211e-07 9.65440989e-08 4.37055441e-08
 3.63120137e-07 8.53379589e-08 2.16249035e-07 2.66747662e-07
 1.70118668e-07 1.95429948e-07 7.89863037e-08 7.92078794e-08
 2.57433356e-07 1.24526935e-07 6.70154066e-08 1.04833667e-07
 1.37200914e-07 3.25627312e-08 3.95149442e-08 1.03634598e-07
 1.24795875e-07 4.47360264e-08 1.36842914e-07 4.09994918e-08
 6.49815561e-08 5.98738481e-08 6.18610230e-08 2.69951883e-08
 4.51327864e-08 1.92101211e-07 1.94765576e-07 4.58237253e-08
 4.56190605e-08 3.08979544e-08 3.98144891e-07 3.07254766e-07
 3.18659517e-07 4.54462352e-08 1.28266763e-07 5.15371745e-08
 1.60761616e-07 4.86129466e-08 1.26562270e-07 4.63663632e-08
 5.73763934e-08 5.73855381e-08 6.94263704e-08 5.76535584e-08
 5.91992730e-08 2.67312714e-07 1.68965570e-07 7.16975208e-08
 2.48290952e-08 1.44674587e-07 2.74947691e-08 5.21620791e-07
 6.56222880e-08 1.500469

hello
[1.85860813e-07 5.85555320e-08 4.04125046e-07 6.91017377e-09
 2.20176322e-08 6.75151952e-08 2.27725110e-08 4.11740153e-08
 2.60190447e-08 1.15186891e-08 2.56930761e-08 2.36645121e-08
 2.64557780e-08 1.72143224e-08 6.85623647e-08 1.86173441e-08
 1.31251232e-08 2.68742202e-08 2.35244411e-08 2.69106852e-08
 1.89204865e-08 5.20638892e-08 1.52981983e-08 2.01876190e-07
 4.16043413e-08 3.05250936e-08 1.43366448e-08 4.51011246e-08
 5.00080546e-08 1.09207463e-07 1.13588229e-07 3.87204189e-08
 4.58813787e-08 1.56838258e-08 7.08310779e-08 2.22532854e-08
 1.31950024e-08 8.34978309e-08 1.04262012e-08 8.68481997e-09
 1.21502337e-08 1.29832785e-08 8.60770797e-08 4.34802061e-08
 1.77148934e-07 1.79856077e-08 4.43372926e-07 6.23376195e-07
 1.51762833e-08 4.87106462e-08 2.41518450e-08 3.09873336e-08
 3.98967650e-08 9.50133927e-09 6.37740598e-08 4.73472213e-08
 6.31326724e-09 9.18533605e-09 2.59228621e-08 1.79941644e-08
 4.75645585e-08 4.76098938e-09 4.67023709e-09 7.20787412e-08
 1.35511939e-07 1.

hello
[[[[4.9072022e-09 1.6086526e-08 4.8346886e-09 ... 2.2280049e-09
    3.7658432e-09 1.5716463e-08]
   [5.4901683e-09 1.7395045e-09 7.9072908e-09 ... 8.9039158e-09
    3.8307486e-09 6.8803532e-09]
   [7.2481634e-09 2.8108023e-09 5.6532401e-09 ... 8.9676568e-09
    6.8604322e-09 3.0857075e-09]
   ...
   [1.0489195e-08 2.4593660e-08 1.8224092e-08 ... 4.9495355e-08
    1.1221096e-08 3.5179117e-08]
   [4.0314343e-09 4.0676746e-09 4.9496762e-09 ... 5.2975957e-09
    4.2494701e-09 1.3467756e-09]
   [5.6714624e-09 9.9116013e-09 8.4684002e-09 ... 6.0543357e-09
    4.4526454e-09 5.4243197e-09]]]]
hello
[4.51386217e-09 6.91703139e-09 9.47889234e-09 8.35952285e-09
 2.47476031e-08 1.58601221e-08 1.27863258e-08 1.13286793e-08
 1.23779280e-08 4.58912908e-08 6.17254514e-08 1.12896563e-08
 1.01170317e-08 1.51744644e-08 4.71652539e-09 1.41632967e-07
 2.03167616e-08 8.32195912e-09 6.66126576e-09 1.18685390e-08
 2.38835565e-08 6.74046596e-09 1.49142565e-08 1.22950725e-08
 1.32599975e-08 8.06537770e-09

[4.87393379e-08 1.40357779e-08 1.92794616e-08 3.12660688e-08
 2.52784655e-08 1.82570155e-08 3.89019078e-07 7.61031231e-07
 7.62429622e-08 1.44062710e-07 3.27165594e-07 5.67681404e-08
 2.36373676e-08 6.05743082e-08 2.60070632e-08 5.38556364e-08
 1.72366406e-08 1.13354570e-08 1.80600054e-07 8.65465637e-08
 6.58903545e-08 4.25755857e-08 4.43066767e-07 3.09128652e-08
 2.14978684e-08 7.16993185e-08 7.08934733e-09 3.90024724e-07
 1.44326305e-06 6.88938684e-08 7.11864345e-09 1.34419169e-08
 1.08362372e-07 3.82153118e-08 2.22346273e-07 5.07284845e-08
 6.58045494e-08 2.23503662e-07 2.12845066e-07 4.88113265e-08
 9.99852205e-07 1.54972764e-08 1.09535009e-07 7.60975567e-08
 2.14279066e-08 8.51713100e-09 1.06814667e-07 1.10434648e-07
 4.42782877e-08 5.53107213e-08 8.16773493e-08 1.12740956e-07
 4.59479530e-08 4.31084430e-08 1.23780694e-07 1.98212486e-08
 5.70719791e-08 1.67277096e-06 7.94983190e-09 1.43329274e-07
 3.97364346e-08 1.98599004e-08 3.94940756e-08 4.68376626e-08
 1.76778876e-08 7.069464

4/5 [=======================>......] - ETA: 4s

In [ ]:
#Predict validation data
predictVal = model.predict(x_val, verbose=1)
predictVal = predictVal.reshape(predictVal.shape[0])
#predictTrain = predictTrain.astype('int32')
print(x_val.shape)
print(predictTrain.shape)
print(y_val.shape)
print("predict val")
print(np.round(predictVal))
print("y val")
print(y_val)

In [ ]:
#predict for testing data
predictTest = model.predict(x_test, verbose=1)
predictTest = predictTest.reshape(predictTest.shape[0])
#predictTrain = predictTrain.astype('int32')
print(x_test.shape)
print(predictTest.shape)
print(y_test.shape)
print("predict Test")
print(np.round(predictTest))
print("y Test")
print(y_test)


In [ ]:
# Mae , Mse and R2 for training data
from sklearn.metrics import r2_score, median_absolute_error, mean_squared_error

y_true = y_train
predict = predictTrain

r2 = r2_score(y_true, predict)
mae = median_absolute_error(y_true, predict)
mse = mean_squared_error(y_true, predict)

print("MSE = ",mse)
print("MAE = ",mae)
print("R2 =",r2)

In [ ]:
# Mae , Mse and R2 for validation data
from sklearn.metrics import r2_score, median_absolute_error, mean_squared_error

y_true = y_val
predict = predictVal

r2 = r2_score(y_true, predict)
mae = median_absolute_error(y_true, predict)
mse = mean_squared_error(y_true, predict)

print("MSE = ",mse)
print("MAE = ",mae)
print("R2 =",r2)

In [ ]:
# Mae , Mse and R2 for testing data
from sklearn.metrics import r2_score, median_absolute_error, mean_squared_error

y_true = y_test
predict = predictTest

r2 = r2_score(y_true, predict)
mae = median_absolute_error(y_true, predict)
mse = mean_squared_error(y_true, predict)

print("MSE = ",mse)
print("MAE = ",mae)
print("R2 =",r2)

In [ ]:
N = y_test.shape
x = predictTest
y = y_test
colors = y_test
#area = np.pi * (10 * np.random.rand(162))**2  # 0 to 15 point radii
area = 100
#plt.title("\nXception\n", fontsize=18)
pyplot.xlabel("\nPredicted\n", fontsize=12)
pyplot.ylabel("\nGround Truth\n", fontsize=12)
marker_size=15
#plasma viridis hot
pyplot.scatter(x, y, s=area, c=colors, cmap='cool', alpha=0.5)
#pyplot.gca().set_axis_bgcolor('white')

cbar= pyplot.colorbar()
cbar.set_label("Number of Leaves", labelpad=+1)

pyplot.show()